# EXERCISE 3.10

foldRight はスタックが溢れる可能性がある。（スタックセーフでないと言う、とのこと）

ということで、foldLeftを実装する。

```Scala
def foldLeft[A,B](as: List[A], z:B)(f: (B, A) => B): B
```

In [3]:
sealed trait List[+A]
case object Nil extends List[Nothing]
case class Cons[+A](head: A, tail: List[A]) extends List[A]

object List {
    def sum(ints: List[Int]): Int =
        ints match {
            case Nil => 0
            case Cons(x, xs) => x + sum(xs)
        }
    
    def apply[A](as: A*): List[A] =
        if (as.isEmpty) Nil
        else Cons(as.head, apply(as.tail: _*))
    
    def tail[A](as: List[A]): List[A] =
        as match {
            case Nil => Nil
            case Cons(h, t) => t
        }
    
    def setHead[A](l: List[A], a: A): List[A] =
        l match {
            case Nil => apply(a)
            case Cons(_, t) => Cons(a, t)
        }
    
    def drop[A](l: List[A], n: Int): List[A] =
        if (n <= 0) l
        else l match {
            case Nil => Nil
            case Cons(_, t) => drop(t, n-1)
        }
    
    def dropWhile[A](l: List[A], f: A => Boolean): List[A] =
        l match {
            case Cons(h, t) => if (f(h)) dropWhile(t, f) else l
            case _ => l
        }
    
    def init[A](l: List[A]): List[A] =
        l match {
            case Nil => sys.error("empty")
            case Cons(_,Nil) => Nil
            case Cons(h,t) => Cons(h, init(t))
        }
 
    def foldRight[A,B](as: List[A], z: B)(f: (A, B) => B): B = // Utility functions
        as match {
          case Nil => z
          case Cons(x, xs) => f(x, foldRight(xs, z)(f))
    }
    
    def length[A](as: List[A]): Int =
        foldRight(as, 0)((x,y) => 1 + y)
    
    @annotation.tailrec
    def foldLeft[A,B](as: List[A], z: B)(f: (B, A) => B): B =
        as match {
            case Nil => z
            case Cons(x, xs) => foldLeft(xs, f(z, x))(f)
        }
}

val a = List(1,2,3)
val b= List.foldLeft(a, 0)((_+_))

defined trait List
defined object Nil
defined class Cons
defined object List
a: $user.List[Int] = Cons(1,Cons(2,Cons(3,Nil)))
b: Int = 6

末尾再帰になったようなので、これでいいみたい。

回答はこちら。
```Scala
  /*
  It's common practice to annotate functions you expect to be tail-recursive with the `tailrec` annotation. If the
  function is not tail-recursive, it will yield a compile error, rather than silently compiling the code and resulting
  in greater stack space usage at runtime.
  */
  @annotation.tailrec
  def foldLeft[A,B](l: List[A], z: B)(f: (B, A) => B): B = l match {
    case Nil => z
    case Cons(h,t) => foldLeft(t, f(z,h))(f)
  }
```